# ИД23-1 Маслов АН

## Функция нормализации БД и сравнение sqlite3 с pandas

In [5]:
import pandas as pd
import sqlite3 as sql
# устанавливаем соединение с БД
con = sql.connect('supermarket.db')
# создаем объект-курсор
cur = con.cursor()

original_df = pd.read_csv("supermarket_sales -.csv")
original_df.shape
unique_columns = ['City', 'Product line', 'Payment']
for i in unique_columns:
    unique_array = original_df[i].unique()
    cur.execute(f'''CREATE TABLE IF NOT EXISTS "{i}_table" (ID INTEGER PRIMARY KEY, "{i}" TEXT)''')
    cur.executemany(f'''INSERT INTO "{i}_table" ("{i}") VALUES (?)''', [(j,) for j in unique_array])
    con.commit()

cur.execute('''CREATE TABLE IF NOT EXISTS "supermarket_sales_table" (
            "Invoice ID" TEXT,
            "Branch" TEXT,
            "City" INTEGER,
            "Customer type" TEXT,
            "Gender" TEXT,
            "Product line" INTEGER,
            "Unit price" DOUBLE,
            "Quantility" INTEGER,
            "Tax 5%" DOUBLE,
            "Total" DOUBLE,
            "Date" TEXT,
            "Time" TEXT
            "Payment" INTEGER,
            "Cogs" DOUBLE,
            "Gross margin percentage" DOUBLE,
            "Gross income" DOUBLE,
            "Rating" DOUBLE,
            
            CONSTRAINT fk_city FOREIGN KEY (City) REFERENCES "City_table"(ID),
            CONSTRAINT fk_product_line FOREIGN KEY (Product line) REFERENCES "Product line_table"(ID),
            CONSTRAINT fk_payment FOREIGN KEY (Payment) REFERENCES "Payment_table"(ID)
            )''')

for index, row in original_df.iterrows():
    for i in unique_columns:
        cur.execute(f'''SELECT ID FROM "{i}_table" WHERE "{i}" = ?''', (row[i],))
        id_value = cur.fetchone()[0]
        original_df.at[index, i] = id_value
original_df.to_sql("supermarket_sales_table", con, if_exists='append')
cur.execute('''SELECT * FROM "supermarket_sales_table" LIMIT 5''')
print(cur.fetchall())
cur.close()
con.close()



OperationalError: near "line": syntax error

In [ ]:
original_df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,1,Member,Female,1,74.69,7,26.1415,548.9715,1/5/2019,13:08,1,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,2,Normal,Female,2,15.28,5,3.8200,80.2200,3/8/2019,10:29,2,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,1,Normal,Male,3,46.33,7,16.2155,340.5255,3/3/2019,13:23,3,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,1,Member,Male,1,58.22,8,23.2880,489.0480,1/27/2019,20:33,1,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,1,Normal,Male,4,86.31,7,30.2085,634.3785,2/8/2019,10:37,1,604.17,4.761905,30.2085,5.3
